### Colab Setup

In [ ]:
! pip install -q johnsnowlabs

In [ ]:
from johnsnowlabs import *
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install(refresh_install=True, visual=True, force_browser = True)

In [3]:
from johnsnowlabs import *
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F
import pandas as pd

# Automatically load licenste data and start a session with all jars user has access to
spark = nlp.start(visual = True)  

👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.3.2, 💊Spark-Healthcare==4.3.2, 🕶Spark-OCR==4.3.3, running on ⚡ PySpark==3.1.2


## Read pdf file to the dataframe and display

In [6]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/solution_accelerator_esg_and_rr/CF_Industries.pdf

--2023-03-27 13:45:15--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/solution_accelerator_esg_and_rr/CF_Industries.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12674821 (12M) [application/octet-stream]
Saving to: ‘CF_Industries.pdf’

CF_Industries.pdf   100%[===================>]  12.09M  --.-KB/s    in 0.1s    

2023-03-27 13:45:15 (86.2 MB/s) - ‘CF_Industries.pdf’ saved [12674821/12674821]



In [7]:
pdf_path = './CF_Industries.pdf'

pdf_example_df = spark.read.format("binaryFile").load(pdf_path).cache()

visual.display_pdf(pdf_example_df, limit = 20)

Output hidden; open in https://colab.research.google.com to view.

## Extract text from the pdf files

If your PDF files ınclude selectable texts, you should use following code to get raw text

In [8]:
# If text PDF extract text
pdf_to_text = visual.PdfToText()\
    .setInputCol("content")\
    .setOutputCol("text")\
    .setSplitPage(True)\
    .setExtractCoordinates(True)\
    .setStoreSplittedPdf(True)

pipeline = PipelineModel(stages=[
    pdf_to_text
])



## Run pipeline and show results

In [9]:
result = pipeline.transform(pdf_example_df).cache()
result.show(5)

+--------------------+--------------------+--------+--------------------+--------------------+----------------+---------------+--------------------+---------+-------+
|                path|    modificationTime|  length|                text|           positions|height_dimension|width_dimension|             content|exception|pagenum|
+--------------------+--------------------+--------+--------------------+--------------------+----------------+---------------+--------------------+---------+-------+
|file:/content/CF_...|2023-03-27 13:45:...|12674821|CF Industries
202...|[{[{C, 0, 56.0, 1...|           768.0|         1366.0|[25 50 44 46 2D 3...|     null|      0|
|file:/content/CF_...|2023-03-27 13:45:...|12674821|A Message from Ou...|[{[{A, 1, 55.0, 6...|           768.0|         1366.0|[25 50 44 46 2D 3...|     null|      1|
|file:/content/CF_...|2023-03-27 13:45:...|12674821|OUR BUSINESS AND ...|[{[{O, 2, 56.0, 5...|           768.0|         1366.0|[25 50 44 46 2D 3...|     null|      2

## Display text using pandas dataframe

In [10]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

result.select("text").toPandas().head().style.set_properties(**{'white-space': 'pre-wrap', 'text-align': 'left'})

## Extract text from the **scanned** pdf files

If your PDF files include scanned image, you can use follwing code to get raw text

In [11]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/solution_accelerator_esg_and_rr/environment_policy.pdf

--2023-03-27 13:47:29--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/solution_accelerator_esg_and_rr/environment_policy.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155570 (152K) [application/octet-stream]
Saving to: ‘environment_policy.pdf’

environment_policy. 100%[===================>] 151.92K  --.-KB/s    in 0.02s   

2023-03-27 13:47:29 (6.18 MB/s) - ‘environment_policy.pdf’ saved [155570/155570]



In [12]:
pdf_path = './environment_policy.pdf'

scanned_pdf_df = spark.read.format("binaryFile").load(pdf_path).cache()

visual.display_pdf(scanned_pdf_df)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
# If your pdf file is scanned image, you can extract image or text with this code
pdf_to_image = visual.PdfToImage()\
    .setInputCol("content")\
    .setOutputCol("image")\
    .setKeepInput(True)

# Run OCR
ocr = visual.ImageToText()\
    .setInputCol("image")\
    .setOutputCol("text")\
    .setConfidenceThreshold(60)

pipeline = PipelineModel(stages=[
    pdf_to_image,
    ocr
])

In [14]:
scanned_result = pipeline.transform(scanned_pdf_df).cache()
scanned_result.show(5)

+--------------------+--------------------+------+--------------------+--------------------+-----------+-------+-----------+-----------------+---------+--------------------+--------------------+
|                path|    modificationTime|length|             content|               image|total_pages|pagenum|documentnum|       confidence|exception|                text|           positions|
+--------------------+--------------------+------+--------------------+--------------------+-----------+-------+-----------+-----------------+---------+--------------------+--------------------+
|file:/content/env...|2023-03-27 13:47:...|155570|[25 50 44 46 2D 3...|{file:/content/en...|          1|      0|          0|80.80283081054688|         |TLScontact Enviro...|[{[{TLScontact En...|
+--------------------+--------------------+------+--------------------+--------------------+-----------+-------+-----------+-----------------+---------+--------------------+--------------------+



In [15]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

scanned_result.select("text").toPandas().head().style.set_properties(**{'white-space': 'pre-wrap', 'text-align': 'left'})

## We will continue `CF Industries`

In [16]:
result.show(3)

+--------------------+--------------------+--------+--------------------+--------------------+----------------+---------------+--------------------+---------+-------+
|                path|    modificationTime|  length|                text|           positions|height_dimension|width_dimension|             content|exception|pagenum|
+--------------------+--------------------+--------+--------------------+--------------------+----------------+---------------+--------------------+---------+-------+
|file:/content/CF_...|2023-03-27 13:45:...|12674821|CF Industries
202...|[{[{C, 0, 56.0, 1...|           768.0|         1366.0|[25 50 44 46 2D 3...|     null|      0|
|file:/content/CF_...|2023-03-27 13:45:...|12674821|A Message from Ou...|[{[{A, 1, 55.0, 6...|           768.0|         1366.0|[25 50 44 46 2D 3...|     null|      1|
|file:/content/CF_...|2023-03-27 13:45:...|12674821|OUR BUSINESS AND ...|[{[{O, 2, 56.0, 5...|           768.0|         1366.0|[25 50 44 46 2D 3...|     null|      2

In [17]:
# This will return a new DF with all the columns + page_num
result = result.withColumn("page_num", F.monotonically_increasing_id()+1)

result.show(3)

+--------------------+--------------------+--------+--------------------+--------------------+----------------+---------------+--------------------+---------+-------+--------+
|                path|    modificationTime|  length|                text|           positions|height_dimension|width_dimension|             content|exception|pagenum|page_num|
+--------------------+--------------------+--------+--------------------+--------------------+----------------+---------------+--------------------+---------+-------+--------+
|file:/content/CF_...|2023-03-27 13:45:...|12674821|CF Industries
202...|[{[{C, 0, 56.0, 1...|           768.0|         1366.0|[25 50 44 46 2D 3...|     null|      0|       1|
|file:/content/CF_...|2023-03-27 13:45:...|12674821|A Message from Ou...|[{[{A, 1, 55.0, 6...|           768.0|         1366.0|[25 50 44 46 2D 3...|     null|      1|       2|
|file:/content/CF_...|2023-03-27 13:45:...|12674821|OUR BUSINESS AND ...|[{[{O, 2, 56.0, 5...|           768.0|         

In [18]:
result = result.select("page_num", "text")

result.show(10, truncate = 80)

+--------+--------------------------------------------------------------------------------+
|page_num|                                                                            text|
+--------+--------------------------------------------------------------------------------+
|       1|             CF Industries
2021  ESG Report
1CF INDUSTRIES  |  2021 ESG REPORT 
|
|       2|A Message from Our CEO
To our Stakeholders:
CF Industries made substantial pr...|
|       3|OUR BUSINESS AND STRATEGY
At our core, CF Industries is a producer of ammonia...|
|       4|ACCOUNTABILITY AND TRANSPARENCY
Our strategy of accelerating the world’s tran...|
|       5|This report details CF Industries’ progress and opportunities within key envi...|
|       6|At CF Industries, our mission is to provide clean energy to feed and fuel 
th...|
|       7|AMMONIA’S ROLE IN SOCIETY 
CF Industries is a leading global manufacturer of ...|
|       8|(1) Other segment products include DEF, urea liquor, nitric acid, aqua

In this dataframe, each row corresponds to each page in the PDF file.

In [19]:
result.write.mode("overwrite").parquet("cf_industries_pages.parquet")

## You can proceed to 03.Responsibility_Reports_Analysis!